# Batting/Fielding HOF Predidctor 

In [169]:
import pandas as pd
import numpy as np
import requests
#from rpy2.robjects import r, pandas2ri
import pyreadr
import matplotlib.pyplot as plt
from pandasql import sqldf
from bs4 import BeautifulSoup
#from html_table_parser import HTMLTableParser
from selenium import webdriver
import time
import datetime
from dateutil.relativedelta import relativedelta
from baseball_scraper import playerid_lookup
from unidecode import unidecode
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
import warnings  
from sklearn.exceptions import UndefinedMetricWarning
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

Ideas:
Could look into final game field in people df and try to come up with an active field if a player hasn't played over x time (years). will have to look into field to see.

Could also calculate the probability of batters and pitchers year to year to try to see when it is in their careers they get tho the HOF thrshold. could put it up on tableau cool viz

In [170]:
# Setting up function for loading Rdata into python from Github Source

def git_load(Name):
    github_url = f'https://raw.githubusercontent.com/an-izq/Pitcher-HOF-Predictor/main/lahman_1871-2023_csv/lahman_1871-2023_csv/{Name}.csv'
    result = pd.read_csv(github_url, encoding = "ISO-8859-1")
    globals()[f'{Name}_df'] = result
    print(f'{Name}_df Success!')
    
# Setting up function to display the correlation in upper/lower triangle. Inspired by pair.panels in R

def corr_func(x, y, **kwargs):
    corr_value = x.corr(y)
    ax = plt.gca()
    ax.annotate(f'Corr: {corr_value:.2f}', xy=(0.5, 0.5), xycoords=ax.transAxes, ha='center', va='center', fontsize=15)
    
# Setting up function to split lists to equal length

def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

In [171]:
# Calling all Tables from Github
table_names = ['People', 'Batting', 'Fielding', 'AwardsPlayers', 'AwardsSharePlayers', 'BattingPost', 'FieldingPost', 'Salaries', 'AllstarFull', 'SeriesPost', 'HallOfFame']
table_names

for x in table_names:
    git_load(x)

People_df Success!
Batting_df Success!
Fielding_df Success!
AwardsPlayers_df Success!
AwardsSharePlayers_df Success!
BattingPost_df Success!
FieldingPost_df Success!
Salaries_df Success!
AllstarFull_df Success!
SeriesPost_df Success!
HallOfFame_df Success!


## People DataFrame

In [172]:
# Looking at People_df
print(People_df.head(5))
People_df.info()
People_df.finalGame.describe()

   ID   playerID  birthYear  birthMonth  birthDay   birthCity birthCountry  \
0   1  aardsda01     1981.0        12.0      27.0      Denver          USA   
1   2  aaronha01     1934.0         2.0       5.0      Mobile          USA   
2   3  aaronto01     1939.0         8.0       5.0      Mobile          USA   
3   4   aasedo01     1954.0         9.0       8.0      Orange          USA   
4   5   abadan01     1972.0         8.0      25.0  Palm Beach          USA   

  birthState  deathYear  deathMonth  ...  nameLast       nameGiven weight  \
0         CO        NaN         NaN  ...   Aardsma     David Allan  215.0   
1         AL     2021.0         1.0  ...     Aaron     Henry Louis  180.0   
2         AL     1984.0         8.0  ...     Aaron      Tommie Lee  190.0   
3         CA        NaN         NaN  ...      Aase  Donald William  190.0   
4         FL        NaN         NaN  ...      Abad   Fausto Andres  184.0   

  height bats throws       debut    bbrefID   finalGame   retroID  


count          20724
unique          9814
top       2022-10-05
freq             478
Name: finalGame, dtype: object

In [173]:
# # Creating an active marker. It's going to be dificult as going off of final game will be tough. trying to do based off date will result in injured players becoming retired, doesnt take into account players retired during that period. It's something I wish I had for analysis but not too big of a deal.
# finalgameDT = pd.to_datetime(People_df['finalGame'])
# finalgameDT = finalgameDT
# fgmax = finalgameDT.max()
# fgfilter = fgmax - relativedelta(years = 1, months= 1)
# len(finalgameDT[finalgameDT > fgfilter])
# People_df.loc[People_df['finalGame'] > fgfilter]

## Batting Dataframe

In [174]:
# Looking at Batting_df
Batting_df.info()
Batting_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113799 entries, 0 to 113798
Data columns (total 24 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   playerID   113799 non-null  object 
 1   yearID     113799 non-null  int64  
 2   stint      113799 non-null  int64  
 3   teamID     113799 non-null  object 
 4   lgID       113062 non-null  object 
 5   G          113799 non-null  int64  
 6   G_batting  1615 non-null    float64
 7   AB         113799 non-null  int64  
 8   R          113799 non-null  int64  
 9   H          113799 non-null  int64  
 10  2B         113799 non-null  int64  
 11  3B         113799 non-null  int64  
 12  HR         113799 non-null  int64  
 13  RBI        113043 non-null  float64
 14  SB         111431 non-null  float64
 15  CS         90257 non-null   float64
 16  BB         113799 non-null  int64  
 17  SO         111699 non-null  float64
 18  IBB        77148 non-null   float64
 19  HBP        110983 non-n

,playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,...,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old
0,aardsda01,2004,1,SFN,NL,11,NaN,0,0,0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,aardsda01,2006,1,CHN,NL,45,NaN,2,0,0,...,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,NaN
2,aardsda01,2007,1,CHA,AL,25,NaN,0,0,0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,aardsda01,2008,1,BOS,AL,47,NaN,1,0,0,...,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,NaN
4,aardsda01,2009,1,SEA,AL,73,NaN,0,0,0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [175]:
# Descriptive stats on Batting_df
Batting_df.describe()

,yearID,stint,G,G_batting,AB,R,H,2B,3B,HR,...,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old
count,113799.000000,113799.000000,113799.000000,1615.000000,113799.000000,113799.000000,113799.000000,113799.000000,113799.000000,113799.000000,...,111431.000000,90257.000000,113799.000000,111699.000000,77148.000000,110983.000000,107731.000000,77695.000000,88357.000000,0.0
mean,1969.631324,1.080774,50.384213,31.424149,137.415513,18.243218,35.841071,6.140151,1.209975,2.874296,...,2.861161,1.138471,12.685814,20.754940,1.008127,1.066695,2.109625,1.003243,2.844800,NaN
std,40.454440,0.290743,46.744082,48.662969,182.992172,27.892612,51.871387,9.600093,2.553320,6.408657,...,7.506894,2.629624,20.482474,29.199861,2.647541,2.318572,4.082782,1.910713,4.640065,NaN
min,1871.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,1939.000000,1.000000,12.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
50%,1979.000000,1.000000,34.000000,1.000000,44.000000,4.000000,8.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,2.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
75%,2004.000000,1.000000,78.000000,49.500000,220.000000,26.000000,54.000000,9.000000,1.000000,2.000000,...,2.000000,1.000000,17.000000,29.000000,1.000000,1.000000,3.000000,1.000000,4.000000,NaN
max,2023.000000,5.000000,165.000000,162.000000,716.000000,198.000000,262.000000,67.000000,36.000000,73.000000,...,138.000000,42.000000,232.000000,223.000000,120.000000,51.000000,67.000000,19.000000,36.000000,NaN


In [176]:
# First year CS (Caught Stealing was used as a statistic) seeing when it is null
print(Batting_df[Batting_df["CS"].isnull()]["yearID"].min())
print(Batting_df[Batting_df["CS"].isnull()]["yearID"].max())
# range of 1876-1950 partially

# First year IBB (Caught Stealing was used as a statistic) seeing when it is null
print(Batting_df[Batting_df["IBB"].isnull()]["yearID"].min())
print(Batting_df[Batting_df["IBB"].isnull()]["yearID"].max())
# range of 1871-1954 partially

# First year SF (Caught Stealing was used as a statistic) seeing when it is null
print(Batting_df[Batting_df["SF"].isnull()]["yearID"].min())
print(Batting_df[Batting_df["SF"].isnull()]["yearID"].max())
# range of 1871-1953 partially

# First year GIDP (Caught Stealing was used as a statistic) seeing when it is null
print(Batting_df[Batting_df["GIDP"].isnull()]["yearID"].min())
print(Batting_df[Batting_df["GIDP"].isnull()]["yearID"].max())
# range of 1876-1938 partially

1876
1950
1871
1954
1871
1953
1876
1938


In [177]:
# joining debut into Batting df and then modifying the debut column
q = '''
SELECT
a.*,
b.debut
FROM
    Batting_df a
LEFT JOIN
    People_df b
        on a.playerID = b.playerID
;
'''
Batting_debut_df = sqldf(q, locals())
# keeping the first four characters of the 'debut' column since that is the year
Batting_debut_df['debut'] = Batting_debut_df['debut'].astype(str).str[:4]
# Printing players who don't have a debut year.... Curious.....
print(Batting_debut_df.loc[Batting_debut_df['debut'] == 'None'])
# Replacing the 'None' values with zeroes since int doesn't take Nan/np.nan or None values
Batting_debut_df.replace({'debut': 'None'}, 0, inplace = True) # {'col' : 'value'}
# Changing debut from str to int
Batting_debut_df['debut'] = Batting_debut_df['debut'].astype(int)
# check
Batting_debut_df.debut.info()

         playerID  yearID  stint teamID  lgID  G  G_batting  AB  R  H  ...  \
22156   crossjo01    1888      1    LS2    AA  1        NaN   1  0  0  ...   
36654   gilgahu01    1875      1    BR2  None  2        NaN   8  2  2  ...   
48974   hyndmji01    1886      1    PH4    AA  1        NaN   4  0  0  ...   
67033   mckenfr01    1874      1    PH2  None  1        NaN   4  0  0  ...   
98503   stineha01    1890      1    PH4    AA  1        NaN   3  0  0  ...   
99694   sulliwi01    1884      1    SLU    UA  1        NaN   5  0  1  ...   
113454  zeihehe01    1886      1    WS8    NL  6        NaN  21  0  0  ...   

         CS  BB    SO  IBB  HBP  SH  SF  GIDP  G_old  debut  
22156   NaN   0   1.0  NaN  0.0 NaN NaN   NaN   None   None  
36654   0.0   0   0.0  NaN  NaN NaN NaN   0.0   None   None  
48974   NaN   0   NaN  NaN  0.0 NaN NaN   NaN   None   None  
67033   0.0   0   1.0  NaN  NaN NaN NaN   0.0   None   None  
98503   NaN   1   NaN  NaN  0.0 NaN NaN   NaN   None   None  
996

In [178]:
# keeping all Batters who have a debut date of greater than or eqaul to 1955
Batting_1955 = Batting_debut_df.loc[Batting_debut_df['debut'] >= 1955]
# getting rid of columns that are not needed
Batting_1955 = Batting_1955.drop(columns=['GIDP', 'debut', 'G_old', 'G_batting'])
print('The rows of the Batting dataframe is: ' + str(len(Batting_1955)))
Batting_1955.rename(columns={'2B':'DB', '3B':'TB'}, inplace=True) # renaming because sql does not take a column starting with a number, will change back.
Batting_1955.columns

The rows of the Batting dataframe is: 73220


Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'G', 'AB', 'R', 'H',
       'DB', 'TB', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH',
       'SF'],
      dtype='object')

In [179]:
# Grouping all the players together by sum or avg in my population
q = '''
SELECT
    playerID,
    sum(G) as G_tot,
    sum(AB) as AB_tot,
    sum(R) as R_tot,
    sum(H) as H_tot,
    sum(DB) as DB_tot,
    sum(TB) as TB_tot,
    sum(HR) as HR_tot,
    sum(RBI) as RBI_tot,
    sum(SB) as SB_tot,
    sum(CS) as CS_tot,
    sum(BB) as BB_tot,
    sum(SO) as SO_tot,
    sum(IBB) as IBB_tot,
    sum(HBP) as HBP_tot,
    sum(SH) as SH_tot,
    sum(SF) as SF_tot
FROM
    Batting_1955
GROUP BY playerID
;
'''
Batting_1955_player_career = sqldf(q, locals())
print('The population of players careers is: ' + str(len(Batting_1955_player_career)))
Batting_1955_player_career.rename(columns={'DB_tot':'2B_tot', 'TB_tot':'3B_tot'}, inplace=True) # renaming because sql does not take a column starting with a number, changing back.
Batting_1955_player_career.head()

The population of players careers is: 11935


,playerID,G_tot,AB_tot,R_tot,H_tot,2B_tot,3B_tot,HR_tot,RBI_tot,SB_tot,CS_tot,BB_tot,SO_tot,IBB_tot,HBP_tot,SH_tot,SF_tot
0,aardsda01,331,4,0,0,0,0,0,0.0,0.0,0.0,0,2.0,0.0,0.0,1.0,0.0
1,aaronto01,437,944,102,216,42,6,13,94.0,9.0,8.0,86,145.0,3.0,0.0,9.0,6.0
2,aasedo01,448,5,0,0,0,0,0,0.0,0.0,0.0,0,3.0,0.0,0.0,0.0,0.0
3,abadan01,15,21,1,2,0,0,0,0.0,0.0,1.0,4,5.0,0.0,0.0,0.0,0.0
4,abadfe01,406,9,0,1,0,0,0,0.0,0.0,0.0,0,5.0,0.0,0.0,0.0,0.0


In [180]:
# Filling in SH and SF with zeros where null since it was not a recorded stat until 1970 so BAOpp counted SH and SF as zero for players before that year.
Batting_1955_player_career.fillna({'SH_tot': 0}, inplace=True)
Batting_1955_player_career.fillna({'SF_tot': 0}, inplace=True)

In [181]:
# Calculating career BA_avg by Dividing the total amount of hits (H) by at bats.
Batting_1955_player_career['BA_avg'] = (Batting_1955_player_career['H_tot'] / (Batting_1955_player_career['AB_tot']))
# Pitching_1955_player_career['BA_avg'].query('playerID == "blylebe01"')
Batting_1955_player_career[['playerID', 'BA_avg']].loc[Batting_1955_player_career['playerID'] == 'aaronto01'] # to verify on baseball reference website

,playerID,BA_avg
1,aaronto01,0.228814


In [182]:
# Calculating career OBP_avg by Dividing the total amount of hits (H) by at bats.
Batting_1955_player_career['OBP'] = ((Batting_1955_player_career['H_tot'] + Batting_1955_player_career['BB_tot'] + Batting_1955_player_career['HBP_tot']) / ((Batting_1955_player_career['AB_tot'] + Batting_1955_player_career['BB_tot'] + Batting_1955_player_career['HBP_tot'] + Batting_1955_player_career['SF_tot'])))
# Pitching_1955_player_career['OBP'].query('playerID == "blylebe01"')
Batting_1955_player_career[['playerID', 'OBP']].loc[Batting_1955_player_career['playerID'] == 'aaronto01'] # to verify on baseball reference website

,playerID,OBP
1,aaronto01,0.291506


In [183]:
# Calculating career SLG by Getting the total amount of singles (subtract 2B, 3B, HR totals from H total) and add 2B, 3B, HR totals (multiply the amount of bases reach ed also to each indiv total) and finally divide by the total AB.
Batting_1955_player_career['SLG'] = (((Batting_1955_player_career['H_tot'] - Batting_1955_player_career['2B_tot'] - Batting_1955_player_career['3B_tot'] - Batting_1955_player_career['HR_tot']) + (2 * Batting_1955_player_career['2B_tot'])  + (3 * Batting_1955_player_career['3B_tot']) + (4 * Batting_1955_player_career['HR_tot'])) / (Batting_1955_player_career['AB_tot']))
# Pitching_1955_player_career['SLG'].query('playerID == "blylebe01"')
Batting_1955_player_career[['playerID', 'SLG']].loc[Batting_1955_player_career['playerID'] == 'aaronto01'] # to verify on baseball reference website

,playerID,SLG
1,aaronto01,0.327331


In [184]:
# Calculating career OPS by adding OBP and SLG.
Batting_1955_player_career['OPS'] = (Batting_1955_player_career['OBP'] + Batting_1955_player_career['SLG'])
# Pitching_1955_player_career['OPS'].query('playerID == "blylebe01"')
Batting_1955_player_career[['playerID', 'OPS']].loc[Batting_1955_player_career['playerID'] == 'aaronto01'] # to verify on baseball reference website

,playerID,OPS
1,aaronto01,0.618836


In [185]:
# looking at the info for the new formatted Dataframe
Batting_1955_player_career.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11935 entries, 0 to 11934
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   playerID  11935 non-null  object 
 1   G_tot     11935 non-null  int64  
 2   AB_tot    11935 non-null  int64  
 3   R_tot     11935 non-null  int64  
 4   H_tot     11935 non-null  int64  
 5   2B_tot    11935 non-null  int64  
 6   3B_tot    11935 non-null  int64  
 7   HR_tot    11935 non-null  int64  
 8   RBI_tot   11935 non-null  float64
 9   SB_tot    11935 non-null  float64
 10  CS_tot    11935 non-null  float64
 11  BB_tot    11935 non-null  int64  
 12  SO_tot    11935 non-null  float64
 13  IBB_tot   11935 non-null  float64
 14  HBP_tot   11935 non-null  float64
 15  SH_tot    11935 non-null  float64
 16  SF_tot    11935 non-null  float64
 17  BA_avg    9692 non-null   float64
 18  OBP       9720 non-null   float64
 19  SLG       9692 non-null   float64
 20  OPS       9692 non-null   fl

In [186]:
# Removing pitchers who have had a rare plate appearance but didn't register an AB
pitcher_bat_df = Batting_1955_player_career[Batting_1955_player_career["OBP"].isnull() | Batting_1955_player_career["BA_avg"].isnull()]
print(len(pitcher_bat_df))
Batting_1955_player_career = Batting_1955_player_career[~Batting_1955_player_career.playerID.isin(pitcher_bat_df['playerID'])]
Batting_1955_player_career.info()

2243
<class 'pandas.core.frame.DataFrame'>
Index: 9692 entries, 0 to 11933
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   playerID  9692 non-null   object 
 1   G_tot     9692 non-null   int64  
 2   AB_tot    9692 non-null   int64  
 3   R_tot     9692 non-null   int64  
 4   H_tot     9692 non-null   int64  
 5   2B_tot    9692 non-null   int64  
 6   3B_tot    9692 non-null   int64  
 7   HR_tot    9692 non-null   int64  
 8   RBI_tot   9692 non-null   float64
 9   SB_tot    9692 non-null   float64
 10  CS_tot    9692 non-null   float64
 11  BB_tot    9692 non-null   int64  
 12  SO_tot    9692 non-null   float64
 13  IBB_tot   9692 non-null   float64
 14  HBP_tot   9692 non-null   float64
 15  SH_tot    9692 non-null   float64
 16  SF_tot    9692 non-null   float64
 17  BA_avg    9692 non-null   float64
 18  OBP       9692 non-null   float64
 19  SLG       9692 non-null   float64
 20  OPS       9692 non-null   flo

In [187]:
# dropping sacrifice hits and sacrifice flys since they were only used to calculate average BAOpp (Opponent Batting Average)
Batting_1955_player_career = Batting_1955_player_career.drop(columns=['SH_tot', 'SF_tot'])
Batting_1955_player_career.columns

Index(['playerID', 'G_tot', 'AB_tot', 'R_tot', 'H_tot', '2B_tot', '3B_tot',
       'HR_tot', 'RBI_tot', 'SB_tot', 'CS_tot', 'BB_tot', 'SO_tot', 'IBB_tot',
       'HBP_tot', 'BA_avg', 'OBP', 'SLG', 'OPS'],
      dtype='object')

In [188]:
Batting_1955_player_career.describe()

,G_tot,AB_tot,R_tot,H_tot,2B_tot,3B_tot,HR_tot,RBI_tot,SB_tot,CS_tot,BB_tot,SO_tot,IBB_tot,HBP_tot,BA_avg,OBP,SLG,OPS
count,9692.000000,9692.000000,9692.000000,9692.000000,9692.000000,9692.000000,9692.000000,9692.000000,9692.000000,9692.000000,9692.000000,9692.000000,9692.000000,9692.000000,9692.000000,9692.000000,9692.000000,9692.000000
mean,368.893211,939.839455,121.822637,242.075733,44.159203,5.658688,25.868655,114.773937,16.989889,7.820780,89.378044,175.589352,7.534977,7.739476,0.180073,0.231202,0.258162,0.489365
std,497.080413,1770.067669,253.287075,487.147783,90.877414,13.393356,64.913563,246.645707,55.082264,19.931219,195.252359,308.985589,22.472256,18.515016,0.119821,0.134411,0.187847,0.310987
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,42.000000,13.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.107143,0.150000,0.125000,0.281037
50%,165.000000,117.000000,10.000000,21.000000,3.000000,0.000000,1.000000,8.000000,0.000000,0.000000,7.000000,34.000000,0.000000,0.000000,0.206717,0.264706,0.285714,0.550737
75%,471.250000,886.000000,102.000000,211.000000,37.000000,4.000000,16.000000,92.250000,7.000000,5.000000,71.000000,191.000000,3.000000,6.000000,0.250000,0.316477,0.377228,0.690602
max,3562.000000,14053.000000,2295.000000,4256.000000,746.000000,166.000000,762.000000,2218.000000,1406.000000,335.000000,2558.000000,2597.000000,688.000000,285.000000,1.000000,1.000000,3.000000,4.000000


Debating on what to do with my population becuase there most definitely are pitchers mixed into this population, who arent dedicated batters but still get a good amount of at bats. Also what to do about the decent sized population (25%) that have at bats below 13. For now I will probably leave them in with the risk on them causing noise and skewing my model. I can comeback and add a pitcher or AB filter later. I'm sure I will get some pitcher that appear as hall of famers with a poor batting career and those will be removed.

In [189]:
# number of batters who have had at least 10 abs
len(Batting_1955_player_career[Batting_1955_player_career['AB_tot'] > 13])

7217

In [190]:
# table to be used in analysis and modeling
Batting_1955_player_career.head(5)

,playerID,G_tot,AB_tot,R_tot,H_tot,2B_tot,3B_tot,HR_tot,RBI_tot,SB_tot,CS_tot,BB_tot,SO_tot,IBB_tot,HBP_tot,BA_avg,OBP,SLG,OPS
0,aardsda01,331,4,0,0,0,0,0,0.0,0.0,0.0,0,2.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
1,aaronto01,437,944,102,216,42,6,13,94.0,9.0,8.0,86,145.0,3.0,0.0,0.228814,0.291506,0.327331,0.618836
2,aasedo01,448,5,0,0,0,0,0,0.0,0.0,0.0,0,3.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
3,abadan01,15,21,1,2,0,0,0,0.0,0.0,1.0,4,5.0,0.0,0.0,0.095238,0.240000,0.095238,0.335238
4,abadfe01,406,9,0,1,0,0,0,0.0,0.0,0.0,0,5.0,0.0,0.0,0.111111,0.111111,0.111111,0.222222


## Fielding

In [191]:
Fielding_df

,playerID,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
0,aardsda01,2004,1,SFN,NL,P,11,0.0,32.0,0,0,0.0,0,NaN,NaN,NaN,NaN,NaN
1,aardsda01,2006,1,CHN,NL,P,45,0.0,159.0,1,5,0.0,1,NaN,NaN,NaN,NaN,NaN
2,aardsda01,2007,1,CHA,AL,P,25,0.0,97.0,2,4,1.0,0,NaN,NaN,NaN,NaN,NaN
3,aardsda01,2008,1,BOS,AL,P,47,0.0,146.0,3,6,0.0,0,NaN,NaN,NaN,NaN,NaN
4,aardsda01,2009,1,SEA,AL,P,73,0.0,214.0,2,5,0.0,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151502,zwilldu01,1915,1,CHF,FL,OF,148,NaN,NaN,356,20,8.0,6,NaN,NaN,NaN,NaN,NaN
151503,zwilldu01,1916,1,CHN,NL,OF,10,NaN,NaN,11,0,0.0,0,NaN,NaN,NaN,NaN,NaN
151504,zychto01,2015,1,SEA,AL,P,13,1.0,55.0,0,3,0.0,0,NaN,NaN,NaN,NaN,NaN
151505,zychto01,2016,1,SEA,AL,P,12,0.0,41.0,0,0,1.0,0,NaN,NaN,NaN,NaN,NaN


In [192]:
# Grouping all the players together by sum or avg in my population
q = '''
SELECT
    playerID,
    sum(GS) as GS_tot,
    sum(InnOuts	) as InnOuts_tot,
    sum(PO) as PO_tot,
    sum(A) as A_tot,
    sum(E) as E_tot,
    sum(DP) as DP_tot,
    sum(PB) as PB_tot,
    sum(WP) as WP_tot,
    sum(SB) as SB_tot,
    sum(CS) as CS_tot,
    sum(ZR) as ZR_tot
FROM
    Fielding_df
GROUP BY playerID
;
'''
Fielding_career_df = sqldf(q, locals())
Fielding_career_df

,playerID,GS_tot,InnOuts_tot,PO_tot,A_tot,E_tot,DP_tot,PB_tot,WP_tot,SB_tot,CS_tot,ZR_tot
0,aardsda01,0.0,1011.0,11,29,3.0,2,NaN,NaN,NaN,NaN,NaN
1,aaronha01,2977.0,78414.0,7436,429,144.0,218,NaN,NaN,NaN,NaN,NaN
2,aaronto01,206.0,6472.0,1317,113,22.0,124,NaN,NaN,NaN,NaN,NaN
3,aasedo01,91.0,3328.0,67,135,13.0,10,NaN,NaN,NaN,NaN,NaN
4,abadan01,4.0,138.0,37,1,1.0,3,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
20524,zupofr01,1.0,114.0,31,1,2.0,0,1.0,1.0,2.0,1.0,0.0
20525,zuvelpa01,136.0,3844.0,267,415,23.0,84,NaN,NaN,NaN,NaN,NaN
20526,zuverge01,31.0,1847.0,45,145,7.0,10,NaN,NaN,NaN,NaN,NaN
20527,zwilldu01,27.0,654.0,755,37,25.0,10,NaN,NaN,NaN,NaN,NaN


In [193]:
# Calculating the Fielding percentage as PO (Put-outs) plus A (Asists) divided by PO plus A plus E (errors)
Fielding_career_df["Fld_pct"] = (Fielding_career_df["PO_tot"] + Fielding_career_df["A_tot"]) / (Fielding_career_df["PO_tot"] + Fielding_career_df["A_tot"] + Fielding_career_df["E_tot"])
Fielding_career_df[['playerID', 'Fld_pct']].loc[Fielding_career_df['playerID'] == 'molinya01']

,playerID,Fld_pct
12694,molinya01,0.994754


In [194]:
# table to be used for analysis and modeling
Fielding_career_df.head(5)

,playerID,GS_tot,InnOuts_tot,PO_tot,A_tot,E_tot,DP_tot,PB_tot,WP_tot,SB_tot,CS_tot,ZR_tot,Fld_pct
0,aardsda01,0.0,1011.0,11,29,3.0,2,NaN,NaN,NaN,NaN,NaN,0.930233
1,aaronha01,2977.0,78414.0,7436,429,144.0,218,NaN,NaN,NaN,NaN,NaN,0.982020
2,aaronto01,206.0,6472.0,1317,113,22.0,124,NaN,NaN,NaN,NaN,NaN,0.984848
3,aasedo01,91.0,3328.0,67,135,13.0,10,NaN,NaN,NaN,NaN,NaN,0.939535
4,abadan01,4.0,138.0,37,1,1.0,3,NaN,NaN,NaN,NaN,NaN,0.974359


## Awards Players Dataframe

In [195]:
# AwardsPlayers_df Exploration
AwardsPlayers_df.info()
#print(AwardsPlayers_df.describe())
AwardsPlayers_df["awardID"].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6797 entries, 0 to 6796
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   playerID  6797 non-null   object
 1   awardID   6797 non-null   object
 2   yearID    6797 non-null   int64 
 3   lgID      6793 non-null   object
 4   tie       64 non-null     object
 5   notes     5127 non-null   object
dtypes: int64(1), object(5)
memory usage: 318.7+ KB


array(['Lou Gehrig Memorial Award', 'Most Valuable Player', 'NLCS MVP',
       'Outstanding DH Award', 'Pitching Triple Crown', 'Platinum Glove',
       'TSN All-Star', 'TSN Major League Player of the Year',
       'TSN Pitcher of the Year', 'TSN Reliever of the Year',
       'World Series MVP', 'Reliever of the Year Award', 'Silver Slugger',
       'ALCS MVP', 'All-Star Game MVP', 'Roberto Clemente Award',
       'Rookie of the Year', 'Gold Glove', 'Babe Ruth Award',
       'Comeback Player of the Year', 'Cy Young Award',
       'Hank Aaron Award', 'Hutch Award', 'SIlver Slugger',
       'Baseball Magazine All-Star', 'Triple Crown', 'TSN Guide MVP',
       'TSN Player of the Year', 'TSN Fireman of the Year',
       'Branch Rickey Award'], dtype=object)

In [196]:
# Setting up from population of players those who had awards and only keeping those awards that contribute to a players career without being repetitive (limiting amount of awards).
q = '''
SELECT 
a.playerID,
count(*) as award_count
FROM 
    AwardsPlayers_df a
INNER JOIN
    Batting_1955_player_career b
        on a.playerID = b.playerID
Where awardID in (
                    'Most Valuable Player', 
                    'NLCS MVP',       
                    'Outstanding DH Award', 
                    'Platinum Glove',
                    'World Series MVP', 
                    'Silver Slugger',
                    'ALCS MVP', 
                    'All-Star Game MVP',
                    'Rookie of the Year', 
                    'Gold Glove', 
                    'Babe Ruth Award',
                    'Cy Young Award',
                    'Hank Aaron Award',
                    'SIlver Slugger', 
                    'Triple Crown'
                )
GROUP BY
a.playerID
;
'''

Batting_1955_awards = sqldf(q, locals())

In [197]:
# table to be used in analysis and modeling
Batting_1955_awards

,playerID,award_count
0,abreubo01,2
1,abreujo02,6
2,acunaro01,6
3,ageeto01,3
4,ahmedni01,2
...,...,...
804,zachrpa01,1
805,zambrca01,3
806,zimmery01,3
807,zitoba01,1


## Awards Share Players Dataframe

In [198]:
# AwardsSharePlayers_df Exploration
# AwardsSharePlayers_df.info()
print(AwardsSharePlayers_df.describe())
print(AwardsSharePlayers_df['awardID'].unique())
#AwardsSharePlayers_df.query('awardID == "Rookie of the Year"')

            yearID    pointsWon    pointsMax   votesFirst
count  7447.000000  7447.000000  7447.000000  7089.000000
mean   1976.090909    45.030751   271.220491     1.671886
std      29.479465    70.287190   128.890813     5.107104
min    1911.000000     0.000000    16.000000     0.000000
25%    1952.000000     4.000000   140.000000     0.000000
50%    1978.000000    13.000000   336.000000     0.000000
75%    2002.000000    55.000000   392.000000     0.000000
max    2023.000000   448.000000   448.000000    32.000000
['Cy Young Award' 'Most Valuable Player' 'Rookie of the Year']


In [199]:
# Setting up the pointsWon_tot field which just counts all of the points won in any 'Cy Young Award' 'Most Valuable Player' 'Rookie of the Year' award 
q = '''
SELECT 
a.awardID,
a.playerID,
sum(a.pointsWon) as pointsWon_tot
FROM 
    AwardsSharePlayers_df a
INNER JOIN
    Batting_1955_player_career b
        on a.playerID = b.playerID
GROUP BY
a.awardID,
a.playerID
;
'''
AwardsSharePlayers_career_df = sqldf(q, locals())
AwardsSharePlayers_career_df

,awardID,playerID,pointsWon_tot
0,Cy Young Award,abbotji01,26
1,Cy Young Award,alcansa01,210
2,Cy Young Award,alexado01,13
3,Cy Young Award,alvarhe01,1
4,Cy Young Award,andujjo01,19
...,...,...,...
2430,Rookie of the Year,zeileto01,4
2431,Rookie of the Year,zieglbr01,1
2432,Rookie of the Year,zimmery01,101
2433,Rookie of the Year,ziskri01,0


In [200]:
# pivoting the AwardsSharePlayers_career_df table
AwardsSharePlayers_career_df_pivot = AwardsSharePlayers_career_df.pivot(index= 'playerID', columns= 'awardID', values= 'pointsWon_tot')
AwardsSharePlayers_career_df_pivot.reset_index(inplace=True)
AwardsSharePlayers_career_df_pivot = AwardsSharePlayers_career_df_pivot.rename_axis(None, axis=1)
AwardsSharePlayers_career_df_pivot.fillna(0, inplace=True)

# table to be used in analysis and modeling
AwardsSharePlayers_career_df_pivot = AwardsSharePlayers_career_df_pivot.rename(columns={'Cy Young Award': 'CYY_totvotes', 'Most Valuable Player': 'MVP_totvotes', 'Rookie of the Year':'ROY_totvotes'})
AwardsSharePlayers_career_df_pivot

,playerID,CYY_totvotes,MVP_totvotes,ROY_totvotes
0,abbotji01,26.0,0.0,10.0
1,abernte02,0.0,8.0,0.0
2,abreubo01,0.0,69.0,0.0
3,abreujo02,0.0,553.0,150.0
4,ackledu01,0.0,0.0,6.0
...,...,...,...,...
1762,zimmery01,0.0,28.0,101.0
1763,ziskri01,0.0,91.0,0.0
1764,zitoba01,114.0,24.0,1.0
1765,zobribe01,0.0,48.0,0.0


## Batting Post Dataframe

In [201]:
# PitchingPost_df Exploration
#PitchingPost_df.info()

BattingPost_df.rename(columns={'2B':'DB', '3B':'TB'}, inplace=True) # renaming because sql does not take a column starting with a number, will change back.

BattingPost_df.describe()
#PitchingPost_df

,yearID,G,AB,R,H,DB,TB,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
count,16857.000000,16857.00000,16857.000000,16857.000000,16857.000000,16857.000000,16857.000000,16857.000000,16857.000000,16857.000000,16656.000000,16857.000000,16857.000000,16857.000000,16656.000000,16656.000000,16656.000000,16656.000000
mean,1992.314172,3.03239,7.361452,0.879872,1.772795,0.311384,0.041822,0.197485,0.821321,0.135908,0.059438,0.699353,1.507089,0.073204,0.063761,0.088016,0.045449,0.139229
std,29.966678,1.85110,8.648706,1.483370,2.535343,0.684276,0.218876,0.539580,1.542620,0.536386,0.270778,1.268487,2.041504,0.328924,0.272230,0.336061,0.223586,0.407829
min,1884.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1981.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2002.000000,3.00000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2014.000000,4.00000,14.000000,1.000000,3.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2023.000000,15.00000,66.000000,13.000000,21.000000,6.000000,4.000000,6.000000,13.000000,15.000000,5.000000,13.000000,17.000000,7.000000,4.000000,5.000000,3.000000,5.000000


In [202]:
# Setting up from population of players those who had postseason appearances
# q = '''
# SELECT 
# a.*
# FROM 
#     BattingPost_df a
# INNER JOIN
#     Batting_1955_player_career b
#         on a.playerID = b.playerID
# ;
# '''
# Batting_1955_post_player = sqldf(q, locals())

# # dropping players who had null ERA or BAOpp due to limited innings pitched
# drop_players = Batting_1955_post_player['playerID'].loc[Batting_1955_post_player['ERA'].isnull() | Batting_1955_post_player['BAOpp'].isnull()].tolist()
# # drop_players[['playerID','ERA', 'BAOpp']]
# print(drop_players)
# Batting_1955_post_player = Batting_1955_post_player[~Batting_1955_post_player.playerID.isin(drop_players)]
# # Pitching_1955_post_player.info()

# Counting the number of Postseason Series and Postseason years that the player appeared in.
q = '''
SELECT 
    a.playerID,
    sum(G) as G_tot,
    sum(AB) as AB_tot,
    sum(R) as R_tot,
    sum(H) as H_tot,
    sum(DB) as DB_tot,
    sum(TB) as TB_tot,
    sum(HR) as HR_tot,
    sum(RBI) as RBI_tot,
    sum(SB) as SB_tot,
    sum(CS) as CS_tot,
    sum(BB) as BB_tot,
    sum(SO) as SO_tot,
    sum(IBB) as IBB_tot,
    sum(HBP) as HBP_tot,
    sum(SH) as SH_tot,
    sum(SF) as SF_tot,
    count(distinct(a.yearID)) as Post_Years_tot,
    count(*) as Post_Series_tot
FROM 
    BattingPost_df a
INNER JOIN
    Batting_1955_player_career b
        on a.playerID = b.playerID
Group by 1
;
'''

Batting_1955_post_Year_Series = sqldf(q, locals())

Batting_1955_post_Year_Series.rename(columns={'DB_tot':'2B_tot', 'TB_tot':'3B_tot'}, inplace=True) # renaming because sql does not take a column starting with a number, changing back.

Batting_1955_post_Year_Series

,playerID,G_tot,AB_tot,R_tot,H_tot,2B_tot,3B_tot,HR_tot,RBI_tot,SB_tot,CS_tot,BB_tot,SO_tot,IBB_tot,HBP_tot,SH_tot,SF_tot,Post_Years_tot,Post_Series_tot
0,aaronto01,1,1,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,1,1
1,aasedo01,2,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,1,1
2,abadfe01,1,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,1,1
3,abbotje01,1,1,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,1,1
4,abbotku01,16,32,0,7,2,0,0,0,0,0.0,1,9,0,0.0,0.0,0.0,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3490,zimmery01,35,117,10,32,7,0,5,17,1,0.0,10,30,0,0.0,0.0,1.0,5,8
3491,ziskri01,6,20,1,8,1,0,0,0,0,0.0,2,5,0,0.0,0.0,0.0,2,2
3492,zitoba01,10,5,0,2,0,0,0,2,0,0.0,0,2,0,0.0,1.0,0.0,6,9
3493,zobribe01,64,228,34,55,17,1,4,14,1,0.0,21,38,2,1.0,1.0,2.0,8,15


In [203]:
# Filling in SH and SF with zeros where null since it was not a recorded stat until 1970 so BAOpp counted SH and SF as zero for players before that year.
Batting_1955_post_Year_Series.fillna({'SH_tot': 0}, inplace=True)
Batting_1955_post_Year_Series.fillna({'SF_tot': 0}, inplace=True)

In [204]:
# Calculating career BA_avg by Dividing the total amount of hits (H) by at bats.
Batting_1955_post_Year_Series['BA_avg_post'] = (Batting_1955_post_Year_Series['H_tot'] / (Batting_1955_post_Year_Series['AB_tot']))
# Pitching_1955_player_career['BA_avg'].query('playerID == "blylebe01"')
Batting_1955_post_Year_Series[['playerID', 'BA_avg_post']].loc[Batting_1955_post_Year_Series['playerID'] == 'ortizda01'] # to verify on baseball reference website

,playerID,BA_avg_post
2336,ortizda01,0.289474


In [205]:
# Calculating career OBP_avg by Dividing the total amount of hits (H) by at bats.
Batting_1955_post_Year_Series['OBP_post'] = ((Batting_1955_post_Year_Series['H_tot'] + Batting_1955_post_Year_Series['BB_tot'] + Batting_1955_post_Year_Series['HBP_tot']) / ((Batting_1955_post_Year_Series['AB_tot'] + Batting_1955_post_Year_Series['BB_tot'] + Batting_1955_post_Year_Series['HBP_tot'] + Batting_1955_post_Year_Series['SF_tot'])))
# Pitching_1955_player_career['OBP'].query('playerID == "blylebe01"')
Batting_1955_post_Year_Series[['playerID', 'OBP_post']].loc[Batting_1955_post_Year_Series['playerID'] == 'ortizda01'] # to verify on baseball reference website

,playerID,OBP_post
2336,ortizda01,0.403794


In [206]:
# Calculating career SLG by Getting the total amount of singles (subtract 2B, 3B, HR totals from H total) and add 2B, 3B, HR totals (multiply the amount of bases reach ed also to each indiv total) and finally divide by the total AB.
Batting_1955_post_Year_Series['SLG_post'] = (((Batting_1955_post_Year_Series['H_tot'] - Batting_1955_post_Year_Series['2B_tot'] - Batting_1955_post_Year_Series['3B_tot'] - Batting_1955_post_Year_Series['HR_tot']) + (2 * Batting_1955_post_Year_Series['2B_tot'])  + (3 * Batting_1955_post_Year_Series['3B_tot']) + (4 * Batting_1955_post_Year_Series['HR_tot'])) / (Batting_1955_post_Year_Series['AB_tot']))
# Pitching_1955_player_career['SLG'].query('playerID == "blylebe01"')
Batting_1955_post_Year_Series[['playerID', 'SLG_post']].loc[Batting_1955_post_Year_Series['playerID'] == 'ortizda01'] # to verify on baseball reference website

,playerID,SLG_post
2336,ortizda01,0.542763


In [211]:
# Calculating career OPS by adding OBP and SLG.
Batting_1955_post_Year_Series['OPS_post'] = (Batting_1955_post_Year_Series['OBP_post'] + Batting_1955_post_Year_Series['SLG_post'])
# Pitching_1955_player_career['OPS'].query('playerID == "blylebe01"')
Batting_1955_post_Year_Series[['playerID', 'OPS_post']].loc[Batting_1955_post_Year_Series['playerID'] == 'ortizda01'] # to verify on baseball reference website

,playerID,OPS_post
2336,ortizda01,0.946557


In [220]:
# table to be used in analysis and modeling
Batting_1955_post_Year_Series.info()
Batting_1955_post_Year_Series.head(5)
Batting_1955_post_Year_Series.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3495 entries, 0 to 3494
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   playerID         3495 non-null   object 
 1   G_tot            3495 non-null   int64  
 2   AB_tot           3495 non-null   int64  
 3   R_tot            3495 non-null   int64  
 4   H_tot            3495 non-null   int64  
 5   2B_tot           3495 non-null   int64  
 6   3B_tot           3495 non-null   int64  
 7   HR_tot           3495 non-null   int64  
 8   RBI_tot          3495 non-null   int64  
 9   SB_tot           3495 non-null   int64  
 10  CS_tot           3495 non-null   float64
 11  BB_tot           3495 non-null   int64  
 12  SO_tot           3495 non-null   int64  
 13  IBB_tot          3495 non-null   int64  
 14  HBP_tot          3495 non-null   float64
 15  SH_tot           3495 non-null   float64
 16  SF_tot           3495 non-null   float64
 17  Post_Years_tot

Index(['playerID', 'G_tot', 'AB_tot', 'R_tot', 'H_tot', '2B_tot', '3B_tot',
       'HR_tot', 'RBI_tot', 'SB_tot', 'CS_tot', 'BB_tot', 'SO_tot', 'IBB_tot',
       'HBP_tot', 'SH_tot', 'SF_tot', 'Post_Years_tot', 'Post_Series_tot',
       'BA_avg_post', 'OBP_post', 'SLG_post', 'OPS_post'],
      dtype='object')

## Allstar Dataframe

In [213]:
# AllstarFull_df Exploration
AllstarFull_df.info()
print(AllstarFull_df.describe())
AllstarFull_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5673 entries, 0 to 5672
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   playerID     5673 non-null   object 
 1   yearID       5673 non-null   int64  
 2   gameNum      5673 non-null   int64  
 3   gameID       5624 non-null   object 
 4   teamID       5673 non-null   object 
 5   lgID         5673 non-null   object 
 6   GP           5673 non-null   int64  
 7   startingPos  1771 non-null   float64
dtypes: float64(1), int64(3), object(4)
memory usage: 354.7+ KB
            yearID      gameNum           GP  startingPos
count  5673.000000  5673.000000  5673.000000  1771.000000
mean   1980.913802     0.092367     0.781773     5.142292
std      26.069192     0.384282     0.413079     2.668517
min    1933.000000     0.000000     0.000000     1.000000
25%    1960.000000     0.000000     1.000000     3.000000
50%    1981.000000     0.000000     1.000000     5.000000
75% 

,playerID,yearID,gameNum,gameID,teamID,lgID,GP,startingPos
0,kalinal01,1957,0,NLS195707090,DET,AL,1,9.0
1,demaejo01,1957,0,NLS195707090,KC1,AL,0,NaN
2,grimbo01,1957,0,NLS195707090,NYA,AL,1,NaN
3,howarel01,1957,0,NLS195707090,NYA,AL,0,NaN
4,loesbi01,1957,0,NLS195707090,BAL,AL,1,NaN


In [214]:
# Allstar Data prep

q = '''
SELECT
a.playerID,
count(*) as allstar_tot
FROM
    AllstarFull_df a
INNER JOIN     
    Batting_1955_player_career b
        on a.playerID = b.playerID
GROUP BY
a.playerID
;
'''

AllstarFull_player_career = sqldf(q, locals())
# table to be used in analysis and modeling
AllstarFull_player_career.head()

,playerID,allstar_tot
0,aasedo01,1
1,abreubo01,2
2,abreujo02,3
3,acunaro01,5
4,ageeto01,2


## Hall of Fame Dataframe

In [215]:
# HallOfFame_df Exploration
HallOfFame_df.info()
print(HallOfFame_df.describe())
# table to be used in analysis and modeling
HallOfFame_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6382 entries, 0 to 6381
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   playerID     6382 non-null   object 
 1   yearid       6382 non-null   int64  
 2   votedBy      6382 non-null   object 
 3   ballots      4593 non-null   float64
 4   needed       4436 non-null   float64
 5   votes        4408 non-null   float64
 6   inducted     6368 non-null   object 
 7   category     6382 non-null   object 
 8   needed_note  1115 non-null   object 
dtypes: float64(3), int64(1), object(5)
memory usage: 448.9+ KB
            yearid      ballots       needed        votes
count  6382.000000  4593.000000  4436.000000  4408.000000
mean   1977.541210   298.910298   226.569883    51.486615
std      24.957022   145.220121   110.139779    85.670255
min    1936.000000    12.000000     9.000000     0.000000
25%    1955.000000   201.000000   151.000000     2.000000
50%    1978.000000  

,playerID,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
0,aaronha01,1982,BBWAA,415.0,312.0,406.0,Y,Player,NaN
1,abbotji01,2005,BBWAA,516.0,387.0,13.0,N,Player,NaN
2,abreubo01,2020,BBWAA,397.0,298.0,22.0,N,Player,NaN
3,abreubo01,2021,BBWAA,401.0,301.0,35.0,N,Player,NaN
4,abreubo01,2022,BBWAA,394.0,296.0,34.0,N,Player,NaN


In [216]:
# checking to see with the population of pitchers greater than 1955 how many are in the HOF
q = '''
SELECT
a.*,
b.inducted
FROM
    Batting_1955_player_career a
LEFT JOIN
    HallOfFame_df b
        on a.playerID = b.playerID
;
'''
Batting_1955_hof = sqldf(q, locals())
# number of pitchers after debuting in 1955 to be inducted into the hall of fame.
print(len(Batting_1955_hof.loc[Batting_1955_hof['inducted'] == 'Y', 'playerID'].unique()))
Batting_1955_hof.loc[Batting_1955_hof['inducted'] == 'Y']
#Batting_1955_hof.loc[Batting_1955_hof['playerID'] == 'bunniji01']

104


,playerID,G_tot,AB_tot,R_tot,H_tot,2B_tot,3B_tot,HR_tot,RBI_tot,SB_tot,CS_tot,BB_tot,SO_tot,IBB_tot,HBP_tot,BA_avg,OBP,SLG,OPS,inducted
176,alomaro01,2379,9073,1508,2724,504,80,210,1134.0,474.0,114.0,1032,1140.0,62.0,50.0,0.300231,0.371245,0.442852,0.814097,Y
262,andersp01,152,477,42,104,9,3,0,34.0,6.0,9.0,42,53.0,1.0,1.0,0.218029,0.281609,0.249476,0.531085,Y
293,aparilu01,2601,10230,1335,2677,394,92,83,791.0,506.0,136.0,736,742.0,22.0,27.0,0.261681,0.310778,0.342522,0.653300,Y
431,bagweje01,2150,7797,1517,2314,488,32,449,1529.0,202.0,78.0,1401,1558.0,155.0,128.0,0.296781,0.407616,0.540336,0.947952,Y
449,baineha01,2830,9908,1299,2866,488,49,384,1628.0,34.0,34.0,1062,1441.0,187.0,14.0,0.289261,0.355680,0.464675,0.820355,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10160,walkela01,1988,6907,1355,2160,471,62,383,1311.0,230.0,76.0,913,1231.0,117.0,138.0,0.312726,0.400224,0.565224,0.965448,Y
10401,willibi01,2488,9350,1410,2711,434,88,426,1475.0,90.0,49.0,1045,1046.0,182.0,43.0,0.289947,0.361431,0.491872,0.853303,Y
10518,winfida01,2973,11003,1669,3110,540,88,465,1833.0,223.0,96.0,1216,1686.0,172.0,25.0,0.282650,0.352622,0.474507,0.827129,Y
10639,yastrca01,3308,11988,1816,3419,646,59,452,1844.0,168.0,116.0,1845,1393.0,190.0,40.0,0.285202,0.379453,0.462045,0.841499,Y


## Combining/Formatting all DataFrames

In [238]:
# Combination of all tables into one
q = '''
SELECT 
    p.*,
    a.award_count,
    b.CYY_totvotes,
    b.MVP_totvotes,
    b.ROY_totvotes,
    --c.W_post_tot,
    --c.L_post_tot,
    --c.G_post_tot,
    --c.GS_post_tot,
    --c.CG_post_tot,
    --c.SHO_post_tot,
    --c.SV_post_tot,
    --c.IPouts_post_tot,
    --c.H_post_tot,
    --c.ER_post_tot,
    --c.HR_post_tot,
    --c.BB_post_tot,
    --c.SO_post_tot,
    --c.BAOpp_post_avg,
    --c.ERA_post_avg,
    --c.IBB_post_tot,
    --c.WP_post_tot,
    --c.HBP_post_tot,
    --c.BK_post_tot,
    --c.BFP_post_tot,
    --c.GF_post_tot,
    --c.R_post_tot,
    c.BA_avg_post,
    c.OPS_post,
    c.Post_Years_tot,
    c.Post_Series_tot,
    d.allstar_tot,
    e.inducted
FROM
    Batting_1955_player_career p
LEFT JOIN
    Batting_1955_awards a
        on p.playerID = a.playerID
LEFT JOIN
    AwardsSharePlayers_career_df_pivot b
        on p.playerID = b.playerID    
LEFT JOIN
    Batting_1955_post_Year_Series c
        on p.playerID = c.playerID   
LEFT JOIN
    AllstarFull_player_career d
        on p.playerID = d.playerID
LEFT JOIN
    (SELECT
        *
        FROM
        HallOfFame_df
        WHERE
        inducted = 'Y') e
        on p.playerID = e.playerID

;
'''
Batting_Career = sqldf(q, locals())
print(len(Batting_Career))
Batting_Career.info()

9692
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9692 entries, 0 to 9691
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   playerID         9692 non-null   object 
 1   G_tot            9692 non-null   int64  
 2   AB_tot           9692 non-null   int64  
 3   R_tot            9692 non-null   int64  
 4   H_tot            9692 non-null   int64  
 5   2B_tot           9692 non-null   int64  
 6   3B_tot           9692 non-null   int64  
 7   HR_tot           9692 non-null   int64  
 8   RBI_tot          9692 non-null   float64
 9   SB_tot           9692 non-null   float64
 10  CS_tot           9692 non-null   float64
 11  BB_tot           9692 non-null   int64  
 12  SO_tot           9692 non-null   float64
 13  IBB_tot          9692 non-null   float64
 14  HBP_tot          9692 non-null   float64
 15  BA_avg           9692 non-null   float64
 16  OBP              9692 non-null   float64
 17  SLG      

In [239]:
Batting_Career[['playerID', 'H_tot', 'BA_avg']].loc[Batting_Career['BA_avg'].isna()]

,playerID,H_tot,BA_avg


In [240]:
# filling in HOF NA's with N
Batting_Career.fillna({'inducted': 'N'}, inplace=True)
Batting_Career['inducted'] = np.where(Batting_Career['inducted'] == 'Y', 1, 0)
print(len(Batting_Career.loc[Batting_Career['inducted'].isnull()]))

# filling in Allstar, awards NA's with 0 
Batting_Career.fillna({'inducted': 0}, inplace=True)
Batting_Career.fillna({'allstar_tot': 0}, inplace=True)
Batting_Career.fillna({'CYY_totvotes':0}, inplace=True)
Batting_Career.fillna({'MVP_totvotes': 0}, inplace=True)
Batting_Career.fillna({'ROY_totvotes': 0}, inplace=True)
Batting_Career.fillna({'award_count': 0}, inplace=True)
Batting_Career.fillna({'Post_Years_tot': 0}, inplace=True)
Batting_Career.fillna({'BA_avg_post': 0}, inplace=True)
Batting_Career.fillna({'OPS_post': 0}, inplace=True)
Batting_Career.fillna({'Post_Series_tot': 0}, inplace=True)


0


In [241]:
Batting_Career.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9692 entries, 0 to 9691
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   playerID         9692 non-null   object 
 1   G_tot            9692 non-null   int64  
 2   AB_tot           9692 non-null   int64  
 3   R_tot            9692 non-null   int64  
 4   H_tot            9692 non-null   int64  
 5   2B_tot           9692 non-null   int64  
 6   3B_tot           9692 non-null   int64  
 7   HR_tot           9692 non-null   int64  
 8   RBI_tot          9692 non-null   float64
 9   SB_tot           9692 non-null   float64
 10  CS_tot           9692 non-null   float64
 11  BB_tot           9692 non-null   int64  
 12  SO_tot           9692 non-null   float64
 13  IBB_tot          9692 non-null   float64
 14  HBP_tot          9692 non-null   float64
 15  BA_avg           9692 non-null   float64
 16  OBP              9692 non-null   float64
 17  SLG           

In [242]:
# Creating am accloades_tot column with the sum of award_count, CYY_totvotes, MVP_totvotes, ROY_totVotes, allstar_tot in attempt to reduce the dimensionality of my database. These are all accolades that these pitchers have collected over the years.
Batting_Career['Accolades_tot'] = Batting_Career['award_count'] + Batting_Career['CYY_totvotes'] + Batting_Career['MVP_totvotes'] + Batting_Career['ROY_totvotes'] + Batting_Career['allstar_tot']
cols = Batting_Career.columns.tolist()
cols

['playerID',
 'G_tot',
 'AB_tot',
 'R_tot',
 'H_tot',
 '2B_tot',
 '3B_tot',
 'HR_tot',
 'RBI_tot',
 'SB_tot',
 'CS_tot',
 'BB_tot',
 'SO_tot',
 'IBB_tot',
 'HBP_tot',
 'BA_avg',
 'OBP',
 'SLG',
 'OPS',
 'award_count',
 'CYY_totvotes',
 'MVP_totvotes',
 'ROY_totvotes',
 'BA_avg_post',
 'OPS_post',
 'Post_Years_tot',
 'Post_Series_tot',
 'allstar_tot',
 'inducted',
 'Accolades_tot']

In [243]:
# formatted datatypes
Batting_Career.dtypes

playerID            object
G_tot                int64
AB_tot               int64
R_tot                int64
H_tot                int64
2B_tot               int64
3B_tot               int64
HR_tot               int64
RBI_tot            float64
SB_tot             float64
CS_tot             float64
BB_tot               int64
SO_tot             float64
IBB_tot            float64
HBP_tot            float64
BA_avg             float64
OBP                float64
SLG                float64
OPS                float64
award_count        float64
CYY_totvotes       float64
MVP_totvotes       float64
ROY_totvotes       float64
BA_avg_post        float64
OPS_post           float64
Post_Years_tot     float64
Post_Series_tot    float64
allstar_tot        float64
inducted             int64
Accolades_tot      float64
dtype: object